In [12]:
import os
import numpy as np

def read_dump_file(filename):
    """Đọc một file dump và trích xuất tọa độ x, y, z và nhiệt độ f_Temp[0]."""
    with open(filename, 'r') as f:
        lines = f.readlines()
    
    # Tìm dòng bắt đầu dữ liệu hạt
    for i, line in enumerate(lines):
        if "ITEM: ATOMS" in line:
            start_idx = i + 1
            break
    else:
        raise ValueError(f"Không tìm thấy 'ITEM: ATOMS' trong file {filename}")
    
    # Trích xuất dữ liệu
    data = []
    for line_num, line in enumerate(lines[start_idx:], start=start_idx):
        parts = line.strip().split()
        if not parts:  # Bỏ qua dòng trống
            continue
        try:
            # Giả định ít nhất có 6 cột: id, type, x, y, z, temp
            if len(parts) < 6:
                raise ValueError(f"Dòng {line_num} trong {filename} có ít hơn 6 cột: {line.strip()}")
            # Chỉ lấy 6 cột đầu tiên nếu có nhiều hơn
            id, type, x, y, z, temp = map(float, parts[:6])
            data.append([x, y, z, temp])
        except ValueError as e:
            print(f"Lỗi tại dòng {line_num} trong {filename}: {e}")
            print(f"Nội dung dòng: {line.strip()}")
            continue
    
    if not data:
        raise ValueError(f"Không có dữ liệu hợp lệ trong file {filename}")
    
    return np.array(data)

def read_all_dump_files(folder_path, timestep_size=1.25e-5):
    """Đọc tất cả file .dump trong thư mục và thêm thời gian dựa trên thứ tự."""
    all_data = []
    
    # Lấy danh sách tất cả file .dump trong thư mục
    dump_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.dump')])
    
    if not dump_files:
        print("Không tìm thấy file .dump nào trong thư mục!")
        return None
    
    # Đọc từng file và gán thời gian
    for i, dump_file in enumerate(dump_files):
        filename = os.path.join(folder_path, dump_file)
        try:
            data = read_dump_file(filename)
            # Thêm cột thời gian dựa trên thứ tự file
            t = np.full((data.shape[0], 1), i * timestep_size)
            data_with_time = np.hstack((data[:, :3], t, data[:, 3:]))  # x, y, z, t, temp
            all_data.append(data_with_time)
            print(f"Đã đọc file: {filename} ({data.shape[0]} hạt)")
        except Exception as e:
            print(f"Lỗi khi đọc file {filename}: {e}")
    
    # Kết hợp tất cả dữ liệu thành một mảng
    if all_data:
        all_data = np.vstack(all_data)
        return {
            "x": all_data[:, 0],      # Tọa độ x
            "y": all_data[:, 1],      # Tọa độ y
            "z": all_data[:, 2],      # Tọa độ z
            "t": all_data[:, 3],      # Thời gian
            "temp": all_data[:, 4]    # Nhiệt độ f_Temp[0]
        }
    else:
        print("Không có dữ liệu nào được đọc thành công!")
        return None

# Sử dụng hàm
folder_path = r"C:\Users\MAY02\Desktop\LIGGGHTS\LIGGGHTS\relax_simulation_Al\relax_Al_8line_1"
timestep_size = 1.25e-5  # Thời gian giữa các file dump (giây)

data = read_all_dump_files(folder_path, timestep_size=timestep_size)

if data is not None:
    # In thông tin cơ bản
    print(f"Tổng số điểm dữ liệu: {len(data['x'])}")
    print(f"Miền x: {data['x'].min():.6f} đến {data['x'].max():.6f}")
    print(f"Miền y: {data['y'].min():.6f} đến {data['y'].max():.6f}")
    print(f"Miền z: {data['z'].min():.6f} đến {data['z'].max():.6f}")
    print(f"Miền t: {data['t'].min():.6f} đến {data['t'].max():.6f}")
    print(f"Miền nhiệt độ: {data['temp'].min():.6f} đến {data['temp'].max():.6f}")

Đã đọc file: C:\Users\MAY02\Desktop\LIGGGHTS\LIGGGHTS\relax_simulation_Al\relax_Al_8line_1\coordtemp0.dump (5000 hạt)
Đã đọc file: C:\Users\MAY02\Desktop\LIGGGHTS\LIGGGHTS\relax_simulation_Al\relax_Al_8line_1\coordtemp100000.dump (5000 hạt)
Đã đọc file: C:\Users\MAY02\Desktop\LIGGGHTS\LIGGGHTS\relax_simulation_Al\relax_Al_8line_1\coordtemp1000000.dump (5000 hạt)
Đã đọc file: C:\Users\MAY02\Desktop\LIGGGHTS\LIGGGHTS\relax_simulation_Al\relax_Al_8line_1\coordtemp10000000.dump (5000 hạt)
Đã đọc file: C:\Users\MAY02\Desktop\LIGGGHTS\LIGGGHTS\relax_simulation_Al\relax_Al_8line_1\coordtemp10100000.dump (5000 hạt)
Đã đọc file: C:\Users\MAY02\Desktop\LIGGGHTS\LIGGGHTS\relax_simulation_Al\relax_Al_8line_1\coordtemp10200000.dump (5000 hạt)
Đã đọc file: C:\Users\MAY02\Desktop\LIGGGHTS\LIGGGHTS\relax_simulation_Al\relax_Al_8line_1\coordtemp10300000.dump (5000 hạt)
Đã đọc file: C:\Users\MAY02\Desktop\LIGGGHTS\LIGGGHTS\relax_simulation_Al\relax_Al_8line_1\coordtemp10400000.dump (5000 hạt)
Đã đọc fil

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Định nghĩa mạng PINN
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(4, 100),  # Input: x, y, z, t
            nn.Tanh(),
            nn.Linear(100, 100),
            nn.Tanh(),
            nn.Linear(100, 100),
            nn.Tanh(),
            nn.Linear(100, 1)   # Output: T
        )
    
    def forward(self, x, y, z, t):
        inputs = torch.cat([x, y, z, t], dim=1)
        return self.net(inputs)

# Hàm tính nguồn nhiệt Q từ quỹ đạo laser trong file input
def compute_heat_source(x, y, z, t, omega=np.pi/0.01):
    # Giả định quỹ đạo laser từ file input (bigloop, myloop1, myloop2)
    timestep_size = 1.25e-5
    total_steps_per_file = 100000  # Từ lệnh dump
    t_steps = t / (timestep_size * total_steps_per_file)  # Chuyển đổi t thành chỉ số file
    
    i = torch.floor(t_steps / 40).int()  # 3 bigloop, mỗi loop ~40 file (20 myloop1 + 20 myloop2)
    loop_step = t_steps % 40
    k = torch.floor(loop_step / 2).int()  # Giả định chia đều cho myloop1 và myloop2
    
    # Tọa độ laser (a, b) từ myloop1
    b = 0.001 * k - 0.01
    a = 0.008 * torch.sin(omega * b) + 0.0105 + 0.028 * (i - 1) + 0.0007 * k
    z_laser = 0.01
    
    # Nguồn nhiệt Gaussian
    sigma = 0.0025  # Bán kính vùng nhiệt từ file input
    Q = 1e6 * torch.exp(-((x - a)**2 + (y - b)**2 + (z - z_laser)**2) / (2 * sigma**2))
    return Q

# Hàm loss từ PDE
def compute_physics_loss(model, x, y, z, t, k=204e5, c=9e6, rho=7800):
    x, y, z, t = [v.requires_grad_(True) for v in [x, y, z, t]]
    T = model(x, y, z, t)
    
    # Tính đạo hàm
    T_t = torch.autograd.grad(T, t, grad_outputs=torch.ones_like(T), create_graph=True)[0]
    T_x = torch.autograd.grad(T, x, grad_outputs=torch.ones_like(T), create_graph=True)[0]
    T_xx = torch.autograd.grad(T_x, x, grad_outputs=torch.ones_like(T_x), create_graph=True)[0]
    T_y = torch.autograd.grad(T, y, grad_outputs=torch.ones_like(T_y), create_graph=True)[0]
    T_yy = torch.autograd.grad(T_y, y, grad_outputs=torch.ones_like(T_y), create_graph=True)[0]
    T_z = torch.autograd.grad(T, z, grad_outputs=torch.ones_like(T_z), create_graph=True)[0]
    T_zz = torch.autograd.grad(T_z, z, grad_outputs=torch.ones_like(T_z), create_graph=True)[0]
    
    # Nguồn nhiệt
    Q = compute_heat_source(x, y, z, t)
    
    # Phương trình nhiệt
    residual = rho * c * T_t - k * (T_xx + T_yy + T_zz) - Q
    return torch.mean(residual**2)

# Hàm loss từ dữ liệu dump (nếu có)
def compute_data_loss(model, x, y, z, t, T_true):
    T_pred = model(x, y, z, t)
    return torch.mean((T_pred - T_true)**2)

# Hàm đọc dữ liệu dump (tùy chọn)
def read_all_dump_files(folder_path, timestep_size=1.25e-5):
    all_data = []
    dump_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.dump')])
    
    for i, dump_file in enumerate(dump_files):
        filename = os.path.join(folder_path, dump_file)
        with open(filename, 'r') as f:
            lines = f.readlines()
        for j, line in enumerate(lines):
            if "ITEM: ATOMS" in line:
                start_idx = j + 1
                break
        data = []
        for line in lines[start_idx:]:
            parts = line.strip().split()
            if len(parts) >= 6:
                id, type, x, y, z, temp = map(float, parts[:6])
                data.append([x, y, z, temp])
        data = np.array(data)
        t = np.full((data.shape[0], 1), i * timestep_size)
        data_with_time = np.hstack((data[:, :3], t, data[:, 3:]))
        all_data.append(data_with_time)
    
    all_data = np.vstack(all_data)
    return {
        "x": torch.tensor(all_data[:, 0], dtype=torch.float32).to(device),
        "y": torch.tensor(all_data[:, 1], dtype=torch.float32).to(device),
        "z": torch.tensor(all_data[:, 2], dtype=torch.float32).to(device),
        "t": torch.tensor(all_data[:, 3], dtype=torch.float32).to(device),
        "temp": torch.tensor(all_data[:, 4], dtype=torch.float32).to(device) * 100  # Điều chỉnh nếu cần
    }

# Hàm huấn luyện
def train(model, epochs=10000, lr=0.001, use_dump_data=False, folder_path=None):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    # Điểm ngẫu nhiên cho PDE
    N_pde = 10000
    x_pde = torch.rand(N_pde, 1, device=device) * 0.1
    y_pde = (torch.rand(N_pde, 1, device=device) * 0.03) - 0.015
    z_pde = torch.rand(N_pde, 1, device=device) * 0.02
    t_pde = torch.rand(N_pde, 1, device=device) * 0.003  # 0 đến 3ms
    
    # Đọc dữ liệu dump nếu sử dụng
    if use_dump_data and folder_path:
        data = read_all_dump_files(folder_path)
        x_data, y_data, z_data, t_data, T_data = data["x"], data["y"], data["z"], data["t"], data["temp"]
        batch_size = 10000
    else:
        x_data, y_data, z_data, t_data, T_data = None, None, None, None, None
    
    # Vòng lặp huấn luyện
    for epoch in range(epochs):
        optimizer.zero_grad()
        
        # Loss từ PDE
        loss_pde = compute_physics_loss(model, x_pde, y_pde, z_pde, t_pde)
        loss = loss_pde
        
        # Loss từ dữ liệu dump (nếu có)
        if use_dump_data and x_data is not None:
            idx = np.random.choice(len(x_data), batch_size, replace=False)
            loss_data = compute_data_loss(model, 
                                        x_data[idx], 
                                        y_data[idx], 
                                        z_data[idx], 
                                        t_data[idx], 
                                        T_data[idx])
            loss = loss_pde + 10 * loss_data  # Trọng số để cân bằng
        
        loss.backward()
        optimizer.step()
        
        if epoch % 500 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item():.6f}, PDE: {loss_pde.item():.6f}" + 
                  (f", Data: {loss_data.item():.6f}" if use_dump_data else ""))

# Khởi tạo và huấn luyện
model = PINN().to(device)

# Thay đổi folder_path thành đường dẫn thư mục chứa file dump của bạn
folder_path = r"C:\Users\MAY02\Desktop\LIGGGHTS\LIGGGHTS\relax_simulation_Al\relax_Al_8line_1"

# Huấn luyện với hoặc không dùng dữ liệu dump
train(model, epochs=10000, use_dump_data=True, folder_path=folder_path)
# Nếu không muốn dùng dữ liệu dump, đặt use_dump_data=False

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.backend as K
import scipy.optimize

# Define the Physics-Informed Neural Network (PINN)
class PINN(keras.Model):`
    def __init__(self, layers):
        super(PINN, self).__init__()
        self.model = self.build_model(layers)
    
    def build_model(self, layers):
        model = keras.Sequential()
        model.add(keras.layers.InputLayer(input_shape=(4,)))  # (x, y, z, t)
        for width in layers[1:-1]:
            model.add(keras.layers.Dense(width, activation='tanh'))
        model.add(keras.layers.Dense(layers[-1], activation=None))  # Output T
        return model
    
    def call(self, x):
        return self.model(x)

# Define heat equation residual
@tf.function
def heat_residual(model, x, y, z, t, rho, cp, k, Q):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch([x, y, z, t])
        inputs = tf.stack([x, y, z, t], axis=1)
        T = model(inputs)
        
        T_t = tape.gradient(T, t)
        T_x = tape.gradient(T, x)
        T_y = tape.gradient(T, y)
        T_z = tape.gradient(T, z)
        
        T_xx = tape.gradient(T_x, x)
        T_yy = tape.gradient(T_y, y)
        T_zz = tape.gradient(T_z, z)
    
    del tape
    
    residual = rho * cp * T_t - k * (T_xx + T_yy + T_zz) - Q
    return residual

# Generate training data (spatial + temporal domain)
def generate_training_data(num_points):
    x = np.random.uniform(0, 0.01, num_points)
    y = np.random.uniform(0, 0.01, num_points)
    z = np.random.uniform(0, 0.005, num_points)
    t = np.random.uniform(0, 0.01, num_points)
    return x, y, z, t

# Physics parameters
rho = 7850      # kg/m³ (steel)
cp = 500        # J/(kg*K)
k = 25          # W/(m*K)
Q = 1e6         # Gaussian laser heat source (simplified)

# Training data
num_points = 5000
x_train, y_train, z_train, t_train = generate_training_data(num_points)

# Convert to TensorFlow tensors
x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
z_train = tf.convert_to_tensor(z_train, dtype=tf.float32)
t_train = tf.convert_to_tensor(t_train, dtype=tf.float32)

# Instantiate and compile the model
pinn = PINN(layers=[4, 50, 50, 50, 1])  # Input 4 (x,y,z,t) -> 3 hidden layers -> Output 1 (T)
optimizer = keras.optimizers.Adam(learning_rate=0.001)

# Custom training loop
def train_step():
    with tf.GradientTape() as tape:
        residuals = heat_residual(pinn, x_train, y_train, z_train, t_train, rho, cp, k, Q)
        loss = tf.reduce_mean(tf.square(residuals))
    gradients = tape.gradient(loss, pinn.trainable_variables)
    optimizer.apply_gradients(zip(gradients, pinn.trainable_variables))
    return loss

# Train the model
epochs = 5000
for epoch in range(epochs):
    loss = train_step()
    if epoch % 500 == 0:
        print(f"Epoch {epoch}, Loss: {loss.numpy()}")

# Save the trained model
pinn.save('pinn_heat_slm.h5')


Epoch 0, Loss: 58769354752.0
Epoch 500, Loss: 712.186279296875
Epoch 1000, Loss: 847.318603515625
Epoch 1500, Loss: 557.3079223632812
Epoch 2000, Loss: 1356828.0
Epoch 2500, Loss: 16987.09765625
Epoch 3000, Loss: 16094.3427734375
Epoch 3500, Loss: 16448.322265625
Epoch 4000, Loss: 16177.8203125
Epoch 4500, Loss: 12546.599609375


NotImplementedError: Saving the model to HDF5 format requires the model to be a Functional model or a Sequential model. It does not work for subclassed models, because such models are defined via the body of a Python method, which isn't safely serializable. Consider saving to the Tensorflow SavedModel format (by setting save_format="tf") or using `save_weights`.

In [2]:

# Save the trained model
pinn.save_weights('pinn_heat_slm_weights.h5')


In [4]:
# Load and use the model for prediction
def predict_temperature(x, y, z, t):
    input_tensor = tf.convert_to_tensor([[x, y, z, t]], dtype=tf.float32)
    return pinn(input_tensor).numpy()[0]

In [5]:
x_test, y_test, z_test, t_test = 0.005, 0.005, 0.002, 0.01
predicted_T = predict_temperature(x_test, y_test, z_test, t_test)
print(f"Predicted temperature at ({x_test}, {y_test}, {z_test}, {t_test}): {predicted_T} K")

Predicted temperature at (0.005, 0.005, 0.002, 0.01): [0.01563277] K


In [1]:
# Re-import necessary libraries after execution state reset
import numpy as np

# Define simulation parameters
timestep_value = 15000  # Given TIMESTEP
timestep_size = 1.25e-10  # Timestep in seconds
omega = 30000  # Laser oscillation frequency (rad/s)

# Compute actual time at TIMESTEP 15000
time_at_15000 = timestep_value * timestep_size

# Compute laser coordinates at this time
a_at_15000 = 27 * time_at_15000 + 0.0075 * np.cos(omega * time_at_15000 + np.pi) + 0.0125
b_at_15000 = 0.0075 * np.sin(omega * time_at_15000 + np.pi)

# Display results
a_at_15000, b_at_15000


(0.00506248710617635, -0.0004216525620487625)

In [7]:

# Load and use the model for prediction
def predict_temperature(x, y, z, v):
    input_tensor = tf.convert_to_tensor([[x, y, z, v]], dtype=tf.float32)
    return float(pinn(input_tensor).numpy()[0])

# Predict temperature around a given point
def predict_surrounding_area(x_center, y_center, z_center, v, step=0.001, grid_size=5):
    predictions = []
    for dx in np.linspace(-step, step, grid_size):
        for dy in np.linspace(-step, step, grid_size):
            for dz in np.linspace(-step, step, grid_size):
                x, y, z = x_center + dx, y_center + dy, z_center + dz
                T_pred = predict_temperature(x, y, z, v)
                predictions.append((x, y, z, v, T_pred))
    return predictions

# Example usage: Predict temperature around a specific point
x_center, y_center, z_center, v = 0.005, 0.005, 0.002, 1.0
predictions_surrounding = predict_surrounding_area(x_center, y_center, z_center, v)

# Print predictions for surrounding area
for p in predictions_surrounding:
    print(f"Surrounding Area - x={p[0]:.4f}, y={p[1]:.4f}, z={p[2]:.4f}, v={p[3]:.2f}, T={p[4]:.2f} K")

Surrounding Area - x=0.0040, y=0.0040, z=0.0010, v=1.00, T=0.26 K
Surrounding Area - x=0.0040, y=0.0040, z=0.0015, v=1.00, T=0.26 K
Surrounding Area - x=0.0040, y=0.0040, z=0.0020, v=1.00, T=0.26 K
Surrounding Area - x=0.0040, y=0.0040, z=0.0025, v=1.00, T=0.26 K
Surrounding Area - x=0.0040, y=0.0040, z=0.0030, v=1.00, T=0.26 K
Surrounding Area - x=0.0040, y=0.0045, z=0.0010, v=1.00, T=0.26 K
Surrounding Area - x=0.0040, y=0.0045, z=0.0015, v=1.00, T=0.26 K
Surrounding Area - x=0.0040, y=0.0045, z=0.0020, v=1.00, T=0.26 K
Surrounding Area - x=0.0040, y=0.0045, z=0.0025, v=1.00, T=0.26 K
Surrounding Area - x=0.0040, y=0.0045, z=0.0030, v=1.00, T=0.26 K
Surrounding Area - x=0.0040, y=0.0050, z=0.0010, v=1.00, T=0.26 K
Surrounding Area - x=0.0040, y=0.0050, z=0.0015, v=1.00, T=0.26 K
Surrounding Area - x=0.0040, y=0.0050, z=0.0020, v=1.00, T=0.26 K
Surrounding Area - x=0.0040, y=0.0050, z=0.0025, v=1.00, T=0.26 K
Surrounding Area - x=0.0040, y=0.0050, z=0.0030, v=1.00, T=0.26 K
Surroundin

In [17]:
a=0

In [18]:
for i in range(num_files):
    a+=1
    

In [19]:
a

241

In [20]:
for i in range(241):  # Assuming 241 time steps
    file_name = f"{file_prefix}{i * timestep_interval}.dump"
    full_path = os.path.join(data_dir, file_name)
    
    if file_name not in existing_files:
        print(f"Skipping missing file: {file_name}")
        continue
    
    with open(full_path, "r") as file:
        lines = file.readlines()
    
    start_reading = False
    for line in lines:
        if "ITEM: ATOMS" in line:
            start_reading = True
            continue
        
        if start_reading:
            parts = line.split()
            if len(parts) == 6:
                x, y, z, temp = map(float, parts[2:6])
                t = i * timestep_interval * timestep_size  # Compute actual time
                data_points.append([x, y, z, t, temp])

NameError: name 'existing_files' is not defined

In [21]:
# Directory containing the dump files
data_dir = r"C:\Users\MAY02\Desktop\SLM_NeuralNetwork\Relax_Al_helix_1_1"
file_prefix = "coord"
timestep_interval = 100000  # Time step between each file
timestep_size = 1.25e-10  # Each step duration in seconds

# Get all existing dump files in the directory
existing_files = set(os.listdir(data_dir))
data_points = []

print("Existing files in directory:")
print(existing_files)

# Load data from existing dump files
for i in range(241):  # Assuming 241 time steps
    file_name = f"{file_prefix}{i * timestep_interval}.dump"
    full_path = os.path.join(data_dir, file_name)
    
    if file_name not in existing_files:
        print(f"Skipping missing file: {file_name}")
        continue
    
    print(f"Reading file: {full_path}")
    
    with open(full_path, "r") as file:
        lines = file.readlines()
    
    start_reading = False
    for line in lines:
        if "ITEM: ATOMS" in line:
            start_reading = True
            continue
        
        if start_reading:
            parts = line.split()
            if len(parts) == 6:
                x, y, z, temp = map(float, parts[2:6])
                t = i * timestep_interval * timestep_size  # Compute actual time
                data_points.append([x, y, z, t, temp])

# Convert data to numpy array
if len(data_points) == 0:
    raise ValueError("No data was loaded from dump files. Please check file paths and formats.")
exp_data = np.array(data_points)

# Ensure exp_data is 2D
if exp_data.ndim == 1:
    raise ValueError("Data array is 1D but expected 2D. Check if data is properly loaded.")

Existing files in directory:
{'coord13700000.dump', 'coord112600000.dump', 'coord122100000.dump', 'coord13500000.dump', 'coord121400000.dump', 'coord112900000.dump', 'coord120300000.dump', 'coord18300000.dump', 'coord18900000.dump', 'coord116200000.dump', 'coord123600000.dump', 'coord18000000.dump', 'coord19400000.dump', 'coord118300000.dump', 'coord112800000.dump', 'coord113800000.dump', 'coord112200000.dump', 'coord116900000.dump', 'coord114100000.dump', 'coord110100000.dump', 'coord1400000.dump', 'coord112700000.dump', 'coord115300000.dump', 'coord110300000.dump', 'coord17000000.dump', 'coord117000000.dump', 'coord19500000.dump', 'coord114300000.dump', 'coord17400000.dump', 'coord123100000.dump', 'coord115700000.dump', 'predicted_temperatures.csv', 'coord119900000.dump', 'coord19700000.dump', 'coord11500000.dump', 'coord110200000.dump', 'myfile_helix.in', 'coord111200000.dump', 'coord119700000.dump', 'coord14100000.dump', 'coord113400000.dump', 'coord116600000.dump', 'coord115800000

In [10]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.backend as K
import scipy.optimize
import matplotlib.pyplot as plt
import os

# Directory containing the dump files
data_dir = "/Relax_Al_helix_1_1"
file_prefix = "coord"
num_files = 241  # Number of dump files
timestep_interval = 100000  # Time step between each file
timestep_size = 1.25e-10  # Each step duration in seconds

# Load data from multiple dump files
data_points = []

for i in range(num_files):
    file_name = f"{data_dir}{file_prefix}{i * timestep_interval}.dump"
    if not os.path.exists(file_name):
        continue
    with open(file_name, "r") as file:
        lines = file.readlines()
    
    start_reading = False
    for line in lines:
        if "ITEM: ATOMS" in line:
            start_reading = True
            continue
        
        if start_reading:
            parts = line.split()
            if len(parts) == 6:
                x, y, z, temp = map(float, parts[2:6])
                t = i * timestep_interval * timestep_size  # Compute actual time
                data_points.append([x, y, z, t, temp])

# Convert data to numpy array
if len(data_points) == 0:
    raise ValueError("No data was loaded from dump files. Please check file paths and formats.")
exp_data = np.array(data_points)

# Ensure exp_data is 2D
if exp_data.ndim == 1:
    raise ValueError("Data array is 1D but expected 2D. Check if data is properly loaded.")

# Define the Physics-Informed Machine Learning (PIML) Model
class PINN(keras.Model):
    def __init__(self, layers):
        super(PINN, self).__init__()
        self.model = self.build_model(layers)
    
    def build_model(self, layers):
        model = keras.Sequential()
        model.add(keras.layers.InputLayer(input_shape=(4,)))  # (x, y, z, t)
        for width in layers[1:-1]:
            model.add(keras.layers.Dense(width, activation='swish'))
        model.add(keras.layers.Dense(layers[-1], activation=None))  # Output T
        return model
    
    def call(self, x):
        return self.model(x)

# Training data (using extracted experimental data)
x_train, y_train, z_train, t_train, T_real = exp_data[:, 0], exp_data[:, 1], exp_data[:, 2], exp_data[:, 3], exp_data[:, 4]

# Convert to TensorFlow tensors
x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
z_train = tf.convert_to_tensor(z_train, dtype=tf.float32)
t_train = tf.convert_to_tensor(t_train, dtype=tf.float32)
T_real = tf.convert_to_tensor(T_real, dtype=tf.float32)

# Instantiate and compile the model
pinn = PINN(layers=[4, 150, 150, 150, 150, 150, 1])  # Increased neurons for better learning
optimizer = keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999)

# Custom training loop using real data
def train_step():
    with tf.GradientTape() as tape:
        inputs = tf.stack([x_train, y_train, z_train, t_train], axis=1)
        T_pred = pinn(inputs)
        loss = tf.reduce_mean(tf.square(T_pred - T_real))  # Loss using experimental labels
    gradients = tape.gradient(loss, pinn.trainable_variables)
    optimizer.apply_gradients(zip(gradients, pinn.trainable_variables))
    return loss

# Train the model
epochs = 50000  # Increased training epochs
for epoch in range(epochs):
    loss = train_step()
    if epoch % 1000 == 0:
        print(f"Epoch {epoch}, Loss: {loss.numpy()}")

# Save the trained model
pinn.save_weights('piml_heat_slm_weights.h5')

# Load and use the model for prediction
def predict_temperature(x, y, z, t):
    input_tensor = tf.convert_to_tensor([[x, y, z, t]], dtype=tf.float32)
    return float(pinn(input_tensor).numpy()[0])

# Predict temperature and visualize
def predict_and_visualize():
    X, Y, T_values = [], [], []
    for data in exp_data:
        x, y, z, t, T_exp = data
        T_pred = predict_temperature(x, y, z, t)
        X.append(x)
        Y.append(y)
        T_values.append(T_pred)
    
    plt.figure(figsize=(6,5))
    plt.scatter(X, Y, c=T_values, cmap='jet')
    plt.colorbar(label='Predicted Temperature (K)')
    plt.xlabel('X Position (m)')
    plt.ylabel('Y Position (m)')
    plt.title('Predicted Temperature Distribution over Time')
    plt.show()

# Example visualization
predict_and_visualize()

ValueError: No data was loaded from dump files. Please check file paths and formats.

In [3]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# 🚀 Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 📂 Folder containing LAMMPS dump files
folder_path = r"C:\Users\MAY02\Desktop\LIGGGHTS\LIGGGHTS\relax_simulation_Al\relax_Al_8line_1"

# 📜 Get all .dump files
dump_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".dump")]
dump_files.sort()

# 📌 Function to read LAMMPS dump files
def read_lammps_dump(file_path):
    with open(file_path, "r") as f:
        lines = f.readlines()
    
    start_idx = next(i for i, line in enumerate(lines) if "ITEM: ATOMS" in line) + 1
    columns = lines[start_idx - 1].split()[2:]
    data = [list(map(float, line.split())) for line in lines[start_idx:]]
    
    return pd.DataFrame(data, columns=columns)

# 📥 Read all dump files
lammps_data = {file: read_lammps_dump(file) for file in dump_files}

# 🚀 1. Combine data into a single DataFrame
training_data = []
for time_step, file in enumerate(dump_files):
    df = lammps_data[file].copy()
    df["time"] = time_step
    training_data.append(df)

train_df = pd.concat(training_data, ignore_index=True)

# 🚀 2. Add material and laser parameters
train_df["youngsModulus"] = 69e8
train_df["poissonsRatio"] = 0.33
train_df["coefficientRestitution"] = 0.3
train_df["coefficientFriction"] = 1.05
train_df["thermalConductivity"] = 204e5
train_df["thermalCapacity"] = 9e6
train_df["P"] = 500
train_df["r"] = 0.0005

# 🚀 3. Add sinusoidal temperature variation parameters
train_df["b"] = 0.001 * train_df["id"] - 0.01
train_df["a"] = 0.008 * np.sin((np.pi / 0.01) * train_df["b"]) + 0.0105 + 0.028 * (train_df["time"] - 1) + 0.0007 * train_df["id"]
train_df["d"] = -0.001 * train_df["id"] + 0.01
train_df["c"] = -0.008 * np.sin((np.pi / 0.01) * train_df["d"]) + 0.0105 + 0.014 + (train_df["time"] - 1) * 0.028 + 0.0007 * train_df["id"]

# 🚀 4. Prepare training data
X_train = train_df[['x', 'y', 'z', 'time', 'a', 'b', 'c', 'd', 'youngsModulus', 'poissonsRatio', 
                    'coefficientRestitution', 'coefficientFriction', 
                    'thermalConductivity', 'thermalCapacity', 'P', 'r']].values
y_train = train_df[['f_Temp[0]']].values

# Move data to GPU
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)

# 🚀 5. Define PINN model
class PINN_Network(nn.Module):
    def __init__(self, input_dim=16, hidden_dim=128, output_dim=1, layers=5):
        super(PINN_Network, self).__init__()
        self.hidden_layers = nn.ModuleList()
        self.hidden_layers.append(nn.Linear(input_dim, hidden_dim))
        
        for _ in range(layers - 1):
            self.hidden_layers.append(nn.Linear(hidden_dim, hidden_dim))
        
        self.hidden_layers.append(nn.Linear(hidden_dim, output_dim))
        self.activation = nn.Tanh()

    def forward(self, x):
        for layer in self.hidden_layers[:-1]:
            x = self.activation(layer(x))
        return self.hidden_layers[-1](x)

# 🚀 6. Train PINN model on GPU
model = PINN_Network().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10000
for epoch in range(num_epochs):
    optimizer.zero_grad()
    predictions = model(X_train_tensor)
    loss = criterion(predictions, y_train_tensor)
    loss.backward()
    optimizer.step()
    
    if epoch % 500 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.6f}")

# 🚀 7. Predict temperatures with the trained model
predictions = model(X_train_tensor).detach().cpu().numpy()
train_df["predicted_Temp"] = predictions

# 🚀 8. Save predictions in a .dump file
output_file_path = os.path.join(folder_path, "predicted_temp_gpu.dump")

with open(output_file_path, "w") as f:
    f.write("ITEM: TIMESTEP\n0\n")
    f.write("ITEM: NUMBER OF ATOMS\n" + str(len(train_df)) + "\n")
    f.write("ITEM: BOX BOUNDS pp pp pp\n0 0.1\n-0.015 0.015\n0 0.02\n")
    f.write("ITEM: ATOMS id type x y z time a b c d f_Temp[0] predicted_Temp youngsModulus poissonsRatio coefficientRestitution coefficientFriction thermalConductivity thermalCapacity P r\n")

    for _, row in train_df.iterrows():
        f.write(f"{int(row['id'])} {int(row['type'])} {row['x']} {row['y']} {row['z']} {row['time']} {row['a']} {row['b']} {row['c']} {row['d']} {row['f_Temp[0]']} {row['predicted_Temp']} {row['youngsModulus']} {row['poissonsRatio']} {row['coefficientRestitution']} {row['coefficientFriction']} {row['thermalConductivity']} {row['thermalCapacity']} {row['P']} {row['r']}\n")

print(f"✅ GPU-based predicted file saved at: {output_file_path}")


Using device: cuda
Epoch 0, Loss: 127389.859375
Epoch 500, Loss: 86023.546875
Epoch 1000, Loss: 59360.515625
Epoch 1500, Loss: 40890.507812
Epoch 2000, Loss: 28782.738281
Epoch 2500, Loss: 21471.314453
Epoch 3000, Loss: 17551.558594
Epoch 3500, Loss: 15780.022461
Epoch 4000, Loss: 15150.930664
Epoch 4500, Loss: 14990.580078
Epoch 5000, Loss: 14964.305664
Epoch 5500, Loss: 14961.883789
Epoch 6000, Loss: 14961.778320
Epoch 6500, Loss: 14961.775391
Epoch 7000, Loss: 14961.775391
Epoch 7500, Loss: 14961.775391
Epoch 8000, Loss: 14961.775391
Epoch 8500, Loss: 14961.775391
Epoch 9000, Loss: 14961.775391
Epoch 9500, Loss: 14961.775391
✅ GPU-based predicted file saved at: C:\Users\MAY02\Desktop\LIGGGHTS\LIGGGHTS\relax_simulation_Al\relax_Al_8line_1\predicted_temp_gpu.dump


In [6]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# 🚀 Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 📂 Folder containing LAMMPS dump files
folder_path = r"C:\Users\MAY02\Desktop\LIGGGHTS\LIGGGHTS\relax_simulation_Al\relax_Al_8line_1"

# 📜 Get all .dump files
dump_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".dump")]
dump_files.sort()

# 📌 Function to read LAMMPS dump files
def read_lammps_dump(file_path):
    with open(file_path, "r") as f:
        lines = f.readlines()
    
    start_idx = next(i for i, line in enumerate(lines) if "ITEM: ATOMS" in line) + 1
    columns = lines[start_idx - 1].split()[2:]
    data = [list(map(float, line.split())) for line in lines[start_idx:]]
    
    return pd.DataFrame(data, columns=columns)

# 📥 Read all dump files
lammps_data = {file: read_lammps_dump(file) for file in dump_files}

# 🚀 1. Combine data into a single DataFrame
training_data = []
for time_step, file in enumerate(dump_files):
    df = lammps_data[file].copy()
    df["time"] = time_step
    training_data.append(df)

train_df = pd.concat(training_data, ignore_index=True)

# 🚀 2. Add material and laser parameters
train_df["youngsModulus"] = 69e8
train_df["poissonsRatio"] = 0.33
train_df["coefficientRestitution"] = 0.3
train_df["coefficientFriction"] = 1.05
train_df["thermalConductivity"] = 204e5
train_df["thermalCapacity"] = 9e6
train_df["P"] = 500
train_df["r"] = 0.0005

# 🚀 3. Add sinusoidal temperature variation parameters
train_df["b"] = 0.001 * train_df["id"] - 0.01
train_df["a"] = 0.008 * np.sin((np.pi / 0.01) * train_df["b"]) + 0.0105 + 0.028 * (train_df["time"] - 1) + 0.0007 * train_df["id"]
train_df["d"] = -0.001 * train_df["id"] + 0.01
train_df["c"] = -0.008 * np.sin((np.pi / 0.01) * train_df["d"]) + 0.0105 + 0.014 + (train_df["time"] - 1) * 0.028 + 0.0007 * train_df["id"]

# 🚀 4. Prepare training data
X_train = train_df[['x', 'y', 'z', 'time', 'a', 'b', 'c', 'd', 'youngsModulus', 'poissonsRatio', 
                    'coefficientRestitution', 'coefficientFriction', 
                    'thermalConductivity', 'thermalCapacity', 'P', 'r']].values
y_train = train_df[['f_Temp[0]']].values

# Move data to GPU in smaller batches
batch_size = 8192  # Reduce memory load
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)

# 🚀 5. Define PINN model
class PINN_Network(nn.Module):
    def __init__(self, input_dim=16, hidden_dim=128, output_dim=1, layers=5):
        super(PINN_Network, self).__init__()
        self.hidden_layers = nn.ModuleList()
        self.hidden_layers.append(nn.Linear(input_dim, hidden_dim))
        
        for _ in range(layers - 1):
            self.hidden_layers.append(nn.Linear(hidden_dim, hidden_dim))
        
        self.hidden_layers.append(nn.Linear(hidden_dim, output_dim))
        self.activation = nn.Tanh()

    def forward(self, x):
        for layer in self.hidden_layers[:-1]:
            x = self.activation(layer(x))
        return self.hidden_layers[-1](x)

# 🚀 6. Define Optimized Physics-Informed Loss Function
def physics_loss(model, X):
    X.requires_grad = True  # Enable autograd tracking
    T_pred = model(X)  # Predicted temperature

    # Compute first derivatives
    grad_T = torch.autograd.grad(T_pred.sum(), X, create_graph=True, allow_unused=True)[0]

    # Compute Laplacian of temperature ∇²T
    alpha = 1e-5  # Thermal diffusivity
    heat_equation = grad_T[:, 0]**2 + grad_T[:, 1]**2 + grad_T[:, 2]**2 - (1 / alpha) * grad_T[:, 3]

    return torch.mean(heat_equation**2)  # Minimize physics error

# 🚀 7. Train PINN with Optimized Memory Usage
model = PINN_Network().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10000
lambda_phys = 0.1  # Physics loss weight

for epoch in range(num_epochs):
    for i in range(0, len(X_train_tensor), batch_size):
        X_batch = X_train_tensor[i:i+batch_size]
        y_batch = y_train_tensor[i:i+batch_size]

        optimizer.zero_grad()
        predictions = model(X_batch)
        
        data_loss = criterion(predictions, y_batch)  # MSE Loss
        phys_loss = physics_loss(model, X_batch)  # Physics-Informed Loss

        total_loss = data_loss + lambda_phys * phys_loss  # Combined Loss
        total_loss.backward()
        optimizer.step()
    
    torch.cuda.empty_cache()  # Free GPU memory

    if epoch % 500 == 0:
        print(f"Epoch {epoch}, Total Loss: {total_loss.item():.6f}, Data Loss: {data_loss.item():.6f}, Physics Loss: {phys_loss.item():.6f}")

# 🚀 8. Predict temperatures with the trained model
predictions = model(X_train_tensor).detach().cpu().numpy()
train_df["predicted_Temp"] = predictions

# 🚀 9. Save predictions in a .dump file
output_file_path = os.path.join(folder_path, "predicted_temp_gpu_optimized.dump")

with open(output_file_path, "w") as f:
    f.write("ITEM: TIMESTEP\n0\n")
    f.write("ITEM: NUMBER OF ATOMS\n" + str(len(train_df)) + "\n")
    f.write("ITEM: BOX BOUNDS pp pp pp\n0 0.1\n-0.015 0.015\n0 0.02\n")
    f.write("ITEM: ATOMS id type x y z time a b c d f_Temp[0] predicted_Temp youngsModulus poissonsRatio coefficientRestitution coefficientFriction thermalConductivity thermalCapacity P r\n")

    for _, row in train_df.iterrows():
        f.write(f"{int(row['id'])} {int(row['type'])} {row['x']} {row['y']} {row['z']} {row['time']} {row['a']} {row['b']} {row['c']} {row['d']} {row['f_Temp[0]']} {row['predicted_Temp']} {row['youngsModulus']} {row['poissonsRatio']} {row['coefficientRestitution']} {row['coefficientFriction']} {row['thermalConductivity']} {row['thermalCapacity']} {row['P']} {row['r']}\n")

print(f"✅ Optimized GPU-based PINN model saved at: {output_file_path}")


Using device: cuda
Epoch 0, Total Loss: 141085.093750, Data Loss: 141085.093750, Physics Loss: 0.000000
Epoch 500, Total Loss: 30749.349609, Data Loss: 30749.349609, Physics Loss: 0.000000
Epoch 1000, Total Loss: 30749.349609, Data Loss: 30749.349609, Physics Loss: 0.000000
Epoch 1500, Total Loss: 30749.349609, Data Loss: 30749.349609, Physics Loss: 0.000000
Epoch 2000, Total Loss: 30749.349609, Data Loss: 30749.349609, Physics Loss: 0.000000
Epoch 2500, Total Loss: 30749.349609, Data Loss: 30749.349609, Physics Loss: 0.000000
Epoch 3000, Total Loss: 30749.349609, Data Loss: 30749.349609, Physics Loss: 0.000000
Epoch 3500, Total Loss: 30749.349609, Data Loss: 30749.349609, Physics Loss: 0.000000
Epoch 4000, Total Loss: 30749.349609, Data Loss: 30749.349609, Physics Loss: 0.000000
Epoch 4500, Total Loss: 30749.349609, Data Loss: 30749.349609, Physics Loss: 0.000000
Epoch 5000, Total Loss: 30749.349609, Data Loss: 30749.349609, Physics Loss: 0.000000
Epoch 5500, Total Loss: 30749.349609,

KeyboardInterrupt: 